In [6]:
import torch
from torch_geometric.data import data

In [7]:
# homework 1
x = torch.tensor([[10, 5], [8, 6], [1, 8], [3, 5]], dtype = torch.float)
y = torch.tensor([1, 0, 0, 1], dtype = torch.float)

edge_index = torch.tensor([[0, 1, 0, 2, 1, 2, 1, 3],
                           [1, 0, 2, 0, 2, 1, 3, 1]], dtype = torch.long)


In [8]:
# homework 2
from torch_geometric.datasets import TUDataset
import pandas as pd

In [9]:
data = TUDataset(root = '.', name = 'PROTEINS', use_node_attr = True).shuffle()

Extracting ./PROTEINS/PROTEINS.zip
Processing...
Done!


In [10]:
print('data length: ', len(data))
print('data # of classes: ', data.num_classes)
print('data # of node features: ', data.num_node_features)

data length:  1113
data # of classes:  2
data # of node features:  4


In [11]:
train_idx, val_idx = int(len(data) * 0.8), int(len(data) * 0.9)
train_graph = data[ : train_idx]
val_graph = data[train_idx : val_idx]
test_graph = data[val_idx : ]
print(train_graph, val_graph, test_graph)

PROTEINS(890) PROTEINS(111) PROTEINS(112)


In [12]:
# homework 3
from torch_geometric.datasets import Planetoid

In [13]:
dataset = Planetoid(root = '/tmp/Cora', name = 'Cora')
dataset

Processing...
Done!


Cora()

In [14]:
print('data length: ', len(dataset))
print('data # of classes: ', dataset.num_classes)
print('data # of node features: ', dataset.num_node_features)

data length:  1
data # of classes:  7
data # of node features:  1433


In [261]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = GCNConv(dataset.num_node_features, 256)
    self.conv2 = GCNConv(256, 16)
    self.conv3 = GCNConv(16, dataset.num_classes)
    self.dropout = 0.5

  def forward(self, data):
    x, edge_index = data.x, data.edge_index

    x = self.conv1(x, edge_index)
    x = F.relu(x)
    # x = F.dropout(x, p = self.dropout, training = self.training)

    x = self.conv2(x, edge_index)
    x = F.relu(x)
    x = F.dropout(x, p = self.dropout, training = self.training)

    x = self.conv3(x, edge_index)

    return F.log_softmax(x, dim = 1)

In [266]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, weight_decay = 5e-4)

model.train()

for epoch in range(200):
  optimizer.zero_grad()
  out = model(data)
  loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
  loss.backward()
  optimizer.step()


model.eval()
pred = model(data).argmax(dim = 1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7950
